In [1]:
import os

import tensorflow as tf
import pandas as pd
import numpy as np
    
from catanatron_gym.envs.catanatron_env import ACTION_SPACE_SIZE
from catanatron_gym.features import get_feature_ordering

2022-01-09 19:34:07.136609: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-09 19:34:07.136629: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
DATASET_PATH = os.path.join("../../../data/ab2-dataset", "main.csv.gzip")
BATCH_SIZE = 32
SHUFFLE_SEED = 1

rows_per_bytes = 6345 / 1209734  # ~0.0052
size = os.path.getsize(DATASET_PATH)  # bytes
estimated_rows = size * rows_per_bytes
print(estimated_rows)

258507.50417033827


In [3]:
data = pd.read_csv(DATASET_PATH, nrows=10, compression='gzip')
data

,F_BANK_BRICK,F_BANK_DEV_CARDS,F_BANK_ORE,F_BANK_SHEEP,F_BANK_WHEAT,F_BANK_WOOD,"F_EDGE(0, 1)_P0_ROAD","F_EDGE(0, 1)_P1_ROAD","F_EDGE(0, 20)_P0_ROAD","F_EDGE(0, 20)_P1_ROAD",...,BT_3692,BT_3693,BT_3694,BT_3695,ACTION,RETURN,DISCOUNTED_RETURN,TOURNAMENT_RETURN,DISCOUNTED_TOURNAMENT_RETURN,VICTORY_POINTS_RETURN
0,19.0,25.0,19.0,19.0,19.0,19.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,110,0.0,0.0,8.0,3.998696,7.944987
1,19.0,25.0,19.0,19.0,19.0,19.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,52,0.0,0.0,8.0,3.998696,7.944987
2,18.0,25.0,19.0,18.0,17.0,17.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,107,0.0,0.0,8.0,3.998696,7.944987
3,18.0,25.0,19.0,18.0,17.0,17.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,45,0.0,0.0,8.0,3.998696,7.944987
4,17.0,25.0,19.0,18.0,17.0,17.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,8.0,3.998696,7.944987
5,17.0,25.0,19.0,18.0,17.0,17.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,289,0.0,0.0,8.0,3.998696,7.944987
6,18.0,25.0,19.0,18.0,17.0,17.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,8.0,3.998696,7.944987
7,19.0,25.0,19.0,18.0,17.0,18.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,50,0.0,0.0,8.0,3.998696,7.944987
8,19.0,25.0,19.0,18.0,17.0,18.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,289,0.0,0.0,8.0,3.998696,7.944987
9,19.0,25.0,18.0,17.0,17.0,18.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0,0.0,0.0,8.0,3.998696,7.944987


In [4]:
# ===== Read Dataset
INPUT_COLUMNS = list(filter(lambda x: x.startswith("F_"), data.columns))
LABEL_COLUMN = "ACTION"
RETURN_COLUMN = "DISCOUNTED_RETURN"
INPUT_SHAPE = (len(INPUT_COLUMNS),)
print(INPUT_COLUMNS)

dataset = tf.data.experimental.make_csv_dataset(
    DATASET_PATH,
    batch_size=BATCH_SIZE,
    num_epochs=1,
    label_name=LABEL_COLUMN,
    select_columns=INPUT_COLUMNS + [LABEL_COLUMN, RETURN_COLUMN],
    compression_type="GZIP",
    shuffle=False,
)

['F_BANK_BRICK', 'F_BANK_DEV_CARDS', 'F_BANK_ORE', 'F_BANK_SHEEP', 'F_BANK_WHEAT', 'F_BANK_WOOD', 'F_EDGE(0, 1)_P0_ROAD', 'F_EDGE(0, 1)_P1_ROAD', 'F_EDGE(0, 20)_P0_ROAD', 'F_EDGE(0, 20)_P1_ROAD', 'F_EDGE(0, 5)_P0_ROAD', 'F_EDGE(0, 5)_P1_ROAD', 'F_EDGE(1, 2)_P0_ROAD', 'F_EDGE(1, 2)_P1_ROAD', 'F_EDGE(1, 6)_P0_ROAD', 'F_EDGE(1, 6)_P1_ROAD', 'F_EDGE(10, 11)_P0_ROAD', 'F_EDGE(10, 11)_P1_ROAD', 'F_EDGE(10, 29)_P0_ROAD', 'F_EDGE(10, 29)_P1_ROAD', 'F_EDGE(11, 12)_P0_ROAD', 'F_EDGE(11, 12)_P1_ROAD', 'F_EDGE(11, 32)_P0_ROAD', 'F_EDGE(11, 32)_P1_ROAD', 'F_EDGE(12, 13)_P0_ROAD', 'F_EDGE(12, 13)_P1_ROAD', 'F_EDGE(13, 14)_P0_ROAD', 'F_EDGE(13, 14)_P1_ROAD', 'F_EDGE(13, 34)_P0_ROAD', 'F_EDGE(13, 34)_P1_ROAD', 'F_EDGE(14, 15)_P0_ROAD', 'F_EDGE(14, 15)_P1_ROAD', 'F_EDGE(14, 37)_P0_ROAD', 'F_EDGE(14, 37)_P1_ROAD', 'F_EDGE(15, 17)_P0_ROAD', 'F_EDGE(15, 17)_P1_ROAD', 'F_EDGE(16, 18)_P0_ROAD', 'F_EDGE(16, 18)_P1_ROAD', 'F_EDGE(16, 21)_P0_ROAD', 'F_EDGE(16, 21)_P1_ROAD', 'F_EDGE(17, 18)_P0_ROAD', 'F_EDGE(17

2022-01-09 19:34:08.642799: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-01-09 19:34:08.687698: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-01-09 19:34:08.687728: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (bcollazo-Erazer-X510): /proc/driver/nvidia/version does not exist
2022-01-09 19:34:08.688384: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
# ===== Select Top 10%
return_column_dataset = tf.data.experimental.make_csv_dataset(
    DATASET_PATH,
    batch_size=BATCH_SIZE,
    num_epochs=1,
    select_columns=[RETURN_COLUMN],
    compression_type="GZIP",
    shuffle=False,
)
print("read dataset")

def collect_returns(state, batch):
    features = batch
    return tf.concat([state, features['DISCOUNTED_RETURN']], 0)

initial_state = []
rewards = return_column_dataset.reduce(initial_state=initial_state, reduce_func=collect_returns)
print(rewards)

top_k = tf.math.top_k(rewards, k=10)
top_k.values.numpy()

read dataset


2022-01-09 19:34:10.803399: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-01-09 19:34:10.838934: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 3492130000 Hz


tf.Tensor([0. 0. 0. ... 0. 0. 0.], shape=(248651,), dtype=float32)


array([0.7397004, 0.7397004, 0.7397004, 0.7397004, 0.7397004, 0.7397004,
       0.7397004, 0.7397004, 0.7397004, 0.7397004], dtype=float32)

In [ ]:
# ===== Build Class Weights
def count(counts, batch):
    features, labels = batch
    for i in range(ACTION_SPACE_SIZE):
        class_i = labels == i
        class_i = tf.cast(class_i, tf.int32)
        counts[f"class_{i}"] += tf.reduce_sum(class_i)
    return counts

initial_state = {f"class_{i}": 0 for i in range(ACTION_SPACE_SIZE)}
counts = dataset.reduce(initial_state=initial_state, reduce_func=count)
counts = np.array([counts[f"class_{i}"].numpy() for i in range(ACTION_SPACE_SIZE)]).astype(np.float32)

fractions = counts/counts.sum()
weights = (1 / counts) * (counts.sum() / ACTION_SPACE_SIZE)
weights[weights == np.inf] = 0  # Drop inf to 0
class_weight = {i: weights[i] for i in range(ACTION_SPACE_SIZE)}
print(class_weight)

# def class_func(features, label):
#     return label

# n = ACTION_SPACE_SIZE
# target_dist = [1.0 / n for i in range(n)]
# resampler = tf.data.experimental.rejection_resample(
#     class_func, target_dist=target_dist)

In [ ]:
def preprocess(batch, label):
    features = tf.stack(
        [tf.cast(tensor, tf.float32) 
         for feature_name, tensor in batch.items() 
         if feature_name in INPUT_COLUMNS and feature_name != RETURN_COLUMN
        ], axis=1
    )
    return features, tf.stack(label)

dataset = dataset.map(preprocess)
    # .unbatch().apply(resampler).batch(BATCH_SIZE)\
    # .map(lambda extra_label, features_and_label: features_and_label)\
dataset

In [ ]:
inputs = tf.keras.Input(shape=INPUT_SHAPE)
dense = tf.keras.layers.Dense(64, activation="relu", dtype='float64')(inputs)
output_dense = tf.keras.layers.Dense(ACTION_SPACE_SIZE, dtype='float64')(dense)
model = tf.keras.Model(inputs=inputs, outputs=output_dense)

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
    metrics=['categorical_accuracy']
)
model.summary()

In [ ]:
model.fit(dataset, epochs=10)

In [ ]:
model.save("data/models/cross-entropy-model")